In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

In [2]:
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True

## Load and prepare data

In [3]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.840B.300d'
config = {
    'max_size': 48,
    'min_freq': 0,
    'batch_size': 32,
    'pretrained_file': pretrained_file
}


In [4]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, config)

In [5]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['sorry,', 'i', 'spent', 'it', 'on', 'myself!', 'harvey', "nichols'", 'hilarious', 'christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '0.18333333333333313'} {'Headline': ['three', 'police', 'officers', 'accused', 'of', 'stealing', '??', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news'}


In [6]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

18

In [7]:
max_len = 0
a = AttractiveData.train_data
for i in range(len(a)):
    if len(a[i].Headline) >= max_len:
        max_len = len(a[i].Headline)
max_len

38

## Start to train

In [8]:
config['timestr'] = time.strftime("%Y%m%d-%H%M%S")
config['save_name'] = 'CNN_LSTM'
config['input_dim'] = len(AttractiveData.TEXT.vocab)
config['embedding_dim'] = 300
config['category_dim'] = len(AttractiveData.CATEGORIES_LABEL.vocab)
config['category_embedding_dim'] = 16
config['hidden_dim'] = 128
config['output_dim'] = 1
config['log_steps'] = 10
config['epochs'] = 50
config['lr'] = {
    'encoder': 3e-5,
    'embedding': 1e-5,
    'linear': 1e-5
}
config['num_layers'] = 2
config['kernel_size'] = 3
config['dropout'] = 0.1

pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([12699, 300])


In [9]:
AttractiveTrainer = AttractiveTrainer(config, AttractiveData.device, AttractiveData.trainloader, pretrained_embeddings)

In [10]:
AttractiveTrainer.model, AttractiveTrainer.config['total_params'], AttractiveTrainer.config['total_learned_params']

(AttractiveNet(
   (embedding): AttractiveEmbedding(
     (token): TokenEmbedding(12699, 300, padding_idx=1)
     (position): PositionalEmbedding()
   )
   (cnn1): Sequential(
     (0): Conv1d(300, 256, kernel_size=(3,), stride=(1,))
     (1): ReLU()
   )
   (cnn2): Sequential(
     (0): Conv1d(256, 128, kernel_size=(3,), stride=(1,))
     (1): ReLU()
   )
   (encoder): LSTM(128, 128, num_layers=2, dropout=0.1, bidirectional=True)
   (linear_output): Linear(in_features=512, out_features=1, bias=True)
 ),
 4798757,
 989057)

In [11]:
AttractiveTrainer.train()

Epoch:   2%|▏         | 1/50 [00:00<00:25,  1.96it/s]
EP_train | avg_loss: 0.5320876301266253 |
Epoch:   4%|▍         | 2/50 [00:01<00:24,  1.94it/s]
EP_train | avg_loss: 0.5313345054164529 |
Epoch:   6%|▌         | 3/50 [00:01<00:24,  1.91it/s]
EP_train | avg_loss: 0.5326740257441998 |
Epoch:   8%|▊         | 4/50 [00:02<00:23,  1.92it/s]
EP_train | avg_loss: 0.5317059168592095 |
Epoch:  10%|█         | 5/50 [00:02<00:23,  1.93it/s]
EP_train | avg_loss: 0.5316830016672611 |
Epoch:  12%|█▏        | 6/50 [00:03<00:22,  1.92it/s]
EP_train | avg_loss: 0.5318275070749223 |
Epoch:  14%|█▍        | 7/50 [00:03<00:22,  1.94it/s]
EP_train | avg_loss: 0.5328773471992463 |
Epoch:  16%|█▌        | 8/50 [00:04<00:21,  1.95it/s]
EP_train | avg_loss: 0.5318045462481678 |
Epoch:  18%|█▊        | 9/50 [00:04<00:21,  1.94it/s]
EP_train | avg_loss: 0.5309870732016861 |
Epoch:  20%|██        | 10/50 [00:05<00:20,  1.92it/s]
EP_train | avg_loss: 0.5324461525306106 |
Epoch:  22%|██▏       | 11/50 [00:05<00

## for classification, not better

In [14]:
# from sklearn.metrics import mean_squared_error
# a = AttractiveTrainer.train_predict
# AttractiveData.LABEL.vocab.itos[int(a[0])], AttractiveTrainer.train_true[0]
# correct = 0
# pred_list = []
# true_list = []
# for i in range(len(a)):
#     pred = AttractiveData.LABEL.vocab.itos[int(a[i])]
#     pred_list.append(float(pred))
#     true = AttractiveData.LABEL.vocab.itos[int(AttractiveTrainer.train_true[i])]
#     true_list.append(float(true))
# mean_squared_error(true_list, pred_list)
# # true_list

0.5601443355119825

## Below is testing

In [12]:
from attractivenet import AttractiveNet
PATH = './model/CNN_LSTM_20201031-222546_0.5331.50'
# load_model = TransformerModel(config).to(AttractiveData.device)
load_model = AttractiveNet(config).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

AttractiveNet(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(12699, 300, padding_idx=1)
    (position): PositionalEmbedding()
  )
  (cnn1): Sequential(
    (0): Conv1d(300, 256, kernel_size=(3,), stride=(1,))
    (1): ReLU()
  )
  (cnn2): Sequential(
    (0): Conv1d(256, 128, kernel_size=(3,), stride=(1,))
    (1): ReLU()
  )
  (encoder): LSTM(128, 128, num_layers=2, dropout=0.1, bidirectional=True)
  (linear_output): Linear(in_features=512, out_features=1, bias=True)
)

In [25]:
def predict_attractive(sentence, category):
    if len(sentence) < config['max_size']:
        sentence += ['0'] * (config['max_size'] - len(sentence))
    else:
        sentence = sentence[:config['max_size']]

    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(1)

    prediction = load_model(tensor_sentence, tensor_category)
    
    return prediction

In [26]:
# train mean = 3.15, test mean = 2.8
predict_list = []
with torch.no_grad():
    for i, sentence in enumerate(AttractiveData.test_data):
        # print(i)
        # print(sentence.Headline)
        prediction = predict_attractive(sentence.Headline, sentence.Category)
        predict_list.append(prediction.item() + 2.8)
        # predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv(config['save_name'] + '.csv', index=False)

In [19]:
# train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
# test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
# for each_test in test_category:
#     if each_test not in train_category:
#         print(each_test)
# print()
# for each_train in train_category:
#     if each_train not in test_category:
#         print(each_train)

## Below just for fun guess

In [27]:
import statistics
from sklearn.metrics import mean_squared_error

In [28]:
# train mean = 3.15, test mean = 2.8
train_list = []
for i, sentence in enumerate(AttractiveData.train_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    # train_list.append(prediction.item() + 3.15)
    train_list.append(prediction.item())
mean_squared_error(pd.read_csv('data/train.csv').sort_values(['ID']).Label.to_list(), train_list), statistics.mean(train_list), statistics.stdev(train_list)

(10.4140600453376, 0.00680579473116143, 0.0013295569592471203)

In [29]:
a = AttractiveData.df_train['Label'].to_list()
statistics.mean(a), statistics.stdev(a)

(0.0004084967320261136, 0.7295015193216009)

In [30]:
statistics.mean(predict_list), statistics.stdev(predict_list)

(2.806901204586029, 0.0014380726065986002)

In [24]:
baseline_list = pd.read_csv('baseline.csv').sort_values(['ID']).Label.to_list()
mean_squared_error(baseline_list, predict_list), statistics.mean(baseline_list), statistics.stdev(baseline_list)

(0.09400285430663444, 2.7156126782757597, 0.29355123275379763)

In [14]:
# LSTM my best
# statistics.mean(predict_list), statistics.stdev(predict_list)

(2.8167915543795683, 0.14611407210842048)

In [21]:
a = pd.read_csv('LSTM_base.csv').Label.to_list()
mean_squared_error(baseline_list, a), statistics.mean(a), statistics.stdev(a)

(0.1347375515605904, 2.8379913731293533, 0.1903582104725371)